In [ ]:
# !pip install fake_useragent
# !pip install selenium

In [3]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
# from tqdm.notebook import tqdm
from threading import Thread
from tqdm import tqdm
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# pip install chromedriver-py

In [11]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.headless = True

card_links = []

for i in tqdm(range(1, 100)):
    with webdriver.Chrome(options=options) as driver:
        driver.get(f'https://luding.ru/collection/wine/?PAGEN_1={i}')
        tree = BeautifulSoup(driver.page_source, 'html.parser')
        card_links.extend(re.findall(r'class="link-absolute" href="(\S*)"', str(tree)))
        time.sleep(0.5)

card_links = set(card_links)
card_links

100%|██████████| 99/99 [53:09<00:00, 32.22s/it]


{'/collection/item/gravitation-pinot-noir-saperavi-kuban-novorossiysk-chateau-pinot--284957/',
 '/collection/item/altra-terra-kagor-bogarve-1915--383319/',
 '/collection/item/temryukskiy-pogreb-muskat--20432/',
 '/collection/item/ya-hochu-odin-plyasat-rose-fler-burlyuk--386734/',
 '/collection/item/5-fincas-emporda-do-reserva-castillo-perelada-gift-box--27139/',
 '/collection/item/baccareto-podere-saraceno-famoso-rubicone-igt-tenuta-santa-lucia--279793/',
 '/collection/item/foja-tonda-casetta-valdadige-doc-albino-armani--372210/',
 '/collection/item/pavao-loureiro-vinho-verde-doc---44634/',
 '/collection/item/mucho-mas-rose-felix-solis-avantis--409357/',
 '/collection/item/luis-felipe-edwards-rose--360587/',
 '/collection/item/krasnaia-gorka-cabernet-sauvignon-kuban-galitsky-galitsky--391787/',
 '/collection/item/vina-jaraba-vino-de-pago-la-jaraba-dop-pago-de-la-jaraba--399452/',
 '/collection/item/alamos-cabernet-sauvignon-mendoza--370453/',
 '/collection/item/la-grola-veronese-igt-al

In [12]:
with open('urls.txt', 'w') as f:
    for card in card_links:
        f.write('https://luding.ru' + card)
        f.write('\n')

In [5]:
with open('urls.txt') as f:
    urls = f.readlines()

In [14]:
for url in urls[:5]:
    print(url)

https://luding.ru/collection/item/gravitation-pinot-noir-saperavi-kuban-novorossiysk-chateau-pinot--284957/

https://luding.ru/collection/item/altra-terra-kagor-bogarve-1915--383319/

https://luding.ru/collection/item/temryukskiy-pogreb-muskat--20432/

https://luding.ru/collection/item/ya-hochu-odin-plyasat-rose-fler-burlyuk--386734/

https://luding.ru/collection/item/5-fincas-emporda-do-reserva-castillo-perelada-gift-box--27139/



In [13]:
df = pd.DataFrame(columns=['Объем', 'Бренд', 'Цена со скидкой',
                           'Цена', 'Тип вина', 'Цвет', 'Сахар',
                           'Страна производителя', 'Регион',
                           'Аппелласьон', 'Производитель',
                           'Категория', 'Бренд / Серия',
                           'Сорт винограда', 'Выдержка в ёмкости',
                           'Содержание алкоголя',
                           'Количество в упаковке', 'Температура подачи'])

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.headless = True

for url in tqdm(urls[:10]):
    with webdriver.Chrome(options=options) as driver:
        driver.get(url)
        tree = BeautifulSoup(driver.page_source, 'html.parser')

    try:
        description = {}
        parent_div = tree.find('div', class_='rdd-body__col-description')
        description['Цена'] = tree.find('span', class_='rdd-price__main').get_text(strip=True)

        if tree.find('span', class_='rdd-price__discount'):
            description['Цена со скидкой'] = tree.find('span', class_='rdd-price__discount').get_text(strip=True)
        else:
            description['Цена со скидкой'] = description['Цена']

        description['Объем'] = parent_div.find('a', class_='rdd-volumes__link').get_text(strip=True)
        description['Бренд'] = parent_div.find('a', class_='rdd-brands__link').get_text(strip=True)

        properties = parent_div.find('div', class_='rdd-properties')
        rows = properties.findAll('div', class_='rdd-properties__group')

        for row in rows:
            names = row.findAll('div', class_='rdd-properties__name-dots')
            values = row.findAll('div', class_='rdd-properties__value')

            for name, value in zip(names, values):
                description[name.get_text(strip=True)] = value.get_text(strip=True)

        df = df._append(description, ignore_index=True)
    except:
        print('Не получилось спарсить товар:', url)

df.head()

 30%|███       | 3/10 [00:28<01:00,  8.68s/it]

Не получилось спарсить товар: https://luding.ru/collection/item/temryukskiy-pogreb-muskat--20432/



100%|██████████| 10/10 [04:31<00:00, 27.20s/it]


,Объем,Бренд,Цена со скидкой,Цена,Тип вина,Цвет,Сахар,Страна производителя,Регион,Аппелласьон,...,Категория,Бренд / Серия,Сорт винограда,Выдержка в ёмкости,Содержание алкоголя,Количество в упаковке,Температура подачи,Подарочная упаковка,BiO,Субрегион
0,0.75,Chateau Pinot,580₽,725₽,Тихое,Красное,Сухое,Россия,Краснодарский край,Кубань. Новороссийск ЗГУ,...,ЗГУ,Chateau PinotГравитация,Пино нуарСаперави,Стальной чан,13%,6,16-18,NaN,NaN,NaN
1,0.75,Bogarve 1915,1 160₽,1 450₽,Тихое,Красное,Сладкое,Испания,NaN,NaN,...,NaN,Bogarve 1915Altra Terra,Гарнача тинторера,Бочка,15%,6,14-16,NaN,NaN,NaN
2,0.75,Бурлюк,337₽,421₽,Тихое,Розовое,Полусухое,Россия,Крым,NaN,...,NaN,NaN,РкацителиМолдова,Стальной чан,10.00% - 11%,6,8-10,NaN,NaN,NaN
3,0.75,Castillo Perelada,2 475₽,3 094₽,Тихое,Красное,Сухое,Испания,Каталония,Emporda DO,...,DO,Castillo Perelada5 Finques,ГарначаКаберне совиньонМерлоСираМонастрельКабе...,Бочка,14%,NaN,16-18,Да,NaN,NaN
4,0.75,Tenuta Santa Lucia,1 762₽,2 202₽,Тихое,Белое,Сухое,Италия,Эмилия-Романья,Rubicone IGT,...,IGT/IGP,Santa Lucia,Фамозо,Стальной чан,12.5%,6,10-12,NaN,Organic,NaN


In [18]:
df = pd.DataFrame(columns=['Объем', 'Бренд', 'Цена со скидкой',
                           'Цена', 'Тип вина', 'Цвет', 'Сахар',
                           'Страна производителя', 'Регион',
                           'Аппелласьон', 'Производитель',
                           'Категория', 'Бренд / Серия',
                           'Сорт винограда', 'Выдержка в ёмкости',
                           'Содержание алкоголя',
                           'Количество в упаковке', 'Температура подачи'])


def process(url):
    global df

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument("--disable-extensions")
    options.headless = True

    with webdriver.Chrome(options=options) as driver:
        driver.get(url)
        tree = BeautifulSoup(driver.page_source, 'html.parser')

    try:
        description = {}
        parent_div = tree.find('div', class_='rdd-body__col-description')
        description['Цена'] = tree.find('span', class_='rdd-price__main').get_text(strip=True)

        if tree.find('span', class_='rdd-price__discount'):
            description['Цена со скидкой'] = tree.find('span', class_='rdd-price__discount').get_text(strip=True)
        else:
            description['Цена со скидкой'] = description['Цена']

        description['Объем'] = parent_div.find('a', class_='rdd-volumes__link').get_text(strip=True)
        description['Бренд'] = parent_div.find('a', class_='rdd-brands__link').get_text(strip=True)

        properties = parent_div.find('div', class_='rdd-properties')
        rows = properties.findAll('div', class_='rdd-properties__group')

        for row in rows:
            names = row.findAll('div', class_='rdd-properties__name-dots')
            values = row.findAll('div', class_='rdd-properties__value')

            for name, value in zip(names, values):
                description[name.get_text(strip=True)] = value.get_text(strip=True)

        df = df._append(description, ignore_index=True)
    except:
        print('Не получилось спарсить товар:', url)


thread_list = []
for url in urls[:10]:
    t = Thread(target=process, args=(url,))
    t.start()
    thread_list.append(t)

for t in tqdm(thread_list):
    t.join()



100%|██████████| 10/10 [02:23<00:00, 14.35s/it]
